
<h1 align="center"><font size="5">COLLABORATIVE FILTERING MODEL-BASED</font></h1>

In [1]:
import pandas as pd
from math import sqrt
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from surprise.model_selection import GridSearchCV
from surprise import Reader, Dataset, SVD
from surprise.model_selection import GridSearchCV, cross_validate
from sklearn.model_selection import train_test_split
import time as t


## Load data 

In [2]:

prepared_data = pd.read_csv('data_preprocessed.csv')

In [3]:

prepared_data.head()

,Unnamed: 0,userId,sceneId,rating
0,0,1,1,1
1,1,1,2,1
2,2,1,3,3
3,3,1,4,3
4,4,1,5,3


In [4]:
#Dropping columns  Unnamed
data = prepared_data.drop(['Unnamed: 0'], 1)
#data=original_data


<hr>

<a id="ref3"></a>
# Collaborative Filtering

### User-Item matrix

In [5]:

movie_matrix = data.pivot(index='userId', columns='sceneId',values='rating').fillna(0).astype(int)

In [6]:
movie_matrix


sceneId,1,2,3,4,5,6,7,8,9,10,11,12
userId,,,,,,,,,,,,
1,1,1,3,3,3,1,4,4,3,3,2,3
2,1,1,3,3,4,0,5,4,4,3,3,2
3,1,1,3,3,3,1,4,4,3,3,2,3
4,1,1,3,3,3,1,4,5,4,4,2,3
5,1,1,3,2,3,1,4,5,3,3,2,3
...,...,...,...,...,...,...,...,...,...,...,...,...
996,2,3,4,3,4,3,3,4,2,2,2,3
997,2,2,3,3,4,3,2,3,3,2,2,4
998,3,2,4,4,4,2,2,3,3,2,1,3


In [7]:
data.groupby("userId")

In [8]:
#Movies not rated by user X
movie_not_rated={}
movie_not_rated_idexes={}
for i,row in movie_matrix.iterrows():
    combine=list(zip(row.index,row.values,row))
    idx_row=[(idx,col) for idx,val,col in combine if val==0]
    indixes=[i[1] for i in idx_row]
    row_names=[i[0] for i in idx_row]
    movie_not_rated_idexes[i]=indixes
    movie_not_rated[i]=row_names

### Select user x

In [9]:
selected_user=2

In [10]:
top_n_users=5

In [11]:
top_n_items=5

### Model Based Collaborative Filtering

In [12]:

movies_to_predict=movie_not_rated.get(selected_user)


In [13]:
my_reccomendations = []

In [14]:
reader = Reader(rating_scale=(1, 5))

data_df = Dataset.load_from_df( data[['userId', 'sceneId', 'rating']], reader = reader )

#### SVD

In [15]:
algo = SVD()
svd_recomendations=[]
now=t.time()

In [16]:
algo.fit(data_df.build_full_trainset())

In [17]:
movies_to_predict

[6]

In [18]:

for movie_id in movies_to_predict:
    svd_recomendations.append((movie_id, algo.predict(uid=selected_user,iid=movie_id).est))
    
svd_rec=pd.DataFrame(svd_recomendations, columns=['sceneId', 'prediction score'])

In [19]:
print("Recommendations for user {0} using SVD :".format(selected_user))
svd_rec.sort_values('prediction score', ascending=False).head(top_n_items)


Recommendations for user 2 using SVD :


,sceneId,prediction score
0,6,1.053516


In [20]:
cross_validate(algo=algo, data=data_df, measures=['RMSE','MAE'], cv=5, verbose=True)
later=t.time()
print(later-now)


Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.7363  0.7482  0.7526  0.7567  0.7422  0.7472  0.0072  
MAE (testset)     0.5859  0.5992  0.6040  0.6087  0.5870  0.5970  0.0091  
Fit time          0.51    0.41    0.42    0.42    0.42    0.44    0.04    
Test time         0.01    0.01    0.01    0.04    0.01    0.02    0.01    
3.374509572982788
